<a href="https://colab.research.google.com/github/Keshvii/MedMCQA_Exp/blob/main/MedDatasetExpLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMS ACCURACY CHECK

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai

# GROQ

# Zero-Shot Prompt: llama3-70b-8192(Acc=68.6)

In [ ]:
import re
import time
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from openai import OpenAI

# --- 1. Setup OpenRouter client ---
GROQ_API_KEY = "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"
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=GROQ_API_KEY
)

# --- 2. System Prompt ---
SYSTEM_PROMPT = (
    "You are a medical examination assistant. "
    "For every batch you will receive exactly <=10 multiple-choice questions (MCQs) in the format:\n\n"
    "Q1: <question>\nA. <option>\nB. <option>\nC. <option>\nD. <option>\n\n"
    "Return ONLY one the answer from A,B,C or D (nothing else should be included in answer), for each question STRICTLY IN FORMAT:\n"
    "Q1: A, Q2: C, ..., Q20: B\n"
    "**IMPORTANT**Do not add explanations or extra text."
)

# --- 3. Ask LLM in Batches of 20 ---
def ask_llm_batch_openrouter(questions_batch, model):
    prompt = ""
    for i, row in enumerate(questions_batch, start=1):
        prompt += (

            f"Q{i}: {row['question']}\n"
            f"A. {row['opa']}\n"
            f"B. {row['opb']}\n"
            f"C. {row['opc']}\n"
            f"D. {row['opd']}\n\n"
        )
    prompt += "Write your answers now:\n"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
    )

    return response.choices[0].message.content.strip()

# --- 4. Parse answer letters A/B/C/D into 0/1/2/3 ---
def parse_batch_answers(response_text, expected_len):
    matches = re.findall(r"Q\d+:\s*([A-D])", response_text, flags=re.IGNORECASE)
    preds = [ord(m.upper()) - 65 for m in matches[:expected_len]]
    return preds

# --- 5. Load Validation Data ---
validation_df = pd.read_csv('/content/drive/MyDrive/MedDataset/validation.csv')
validation_df.info()
# validation_df=validation_df.head(200)

# --- 6. Define Models to Evaluate ---
models = [
    "llama3-70b-8192",
]

# --- 7. Batch Evaluation for Each Model ---
batch_size = 10
results = {}

for model_name in models:
    print(f"\n--- Evaluating {model_name} ---")
    correct, total = 0, 0

    total_batches = (len(validation_df) + batch_size - 1) // batch_size
    for i in tqdm(range(0, len(validation_df), batch_size), desc=f"{model_name} Progress", total=total_batches):

        batch_df = validation_df.iloc[i:i + batch_size]
        golds = batch_df["cop"].astype(int).map({0:'A', 1:'B', 2:'C', 3:'D'}).tolist()
        try:

            raw_response = ask_llm_batch_openrouter(batch_df.to_dict("records"), model=model_name)
            matches = re.findall(r"Q\d+:\s*([A-D])", raw_response, flags=re.IGNORECASE)

            if len(matches) != len(golds):
                print(raw_response)
                print(f"Batch {i//batch_size + 1}: Expected {len(golds)} answers, got {len(matches)}. Skipping.")
                continue

            preds = [m.upper() for m in matches[:len(golds)]]
            correct += sum([int(p == g) for p, g in zip(preds, golds)])
            total += len(golds)

        except Exception as e:
            print(f"Error in batch {i//batch_size + 1} for model {model_name}: {e}")
            continue

    if total > 0:
        acc = correct / total
        results[model_name] = acc
        print(f"Accuracy for {model_name}: {acc:.3f} ({correct}/{total})")
    else:
        print(f"No batches processed for {model_name}")

# --- 8. Summary ---
print("\n=== Final Model Accuracies ===")
for model, acc in results.items():
    print(f"{model}: {acc:.3f}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4183 entries, 0 to 4182
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            4183 non-null   object
 1   question      4183 non-null   object
 2   opa           4183 non-null   object
 3   opb           4183 non-null   object
 4   opc           4182 non-null   object
 5   opd           4143 non-null   object
 6   cop           4183 non-null   int64 
 7   choice_type   4183 non-null   object
 8   exp           2206 non-null   object
 9   subject_name  4183 non-null   object
 10  topic_name    423 non-null    object
dtypes: int64(1), object(10)
memory usage: 359.6+ KB

--- Evaluating llama3-70b-8192 ---


llama3-70b-8192 Progress:   0%|          | 0/419 [00:00<?, ?it/s]

Error in batch 118 for model llama3-70b-8192: Connection error.
Error in batch 197 for model llama3-70b-8192: no healthy upstream
Error in batch 198 for model llama3-70b-8192: no healthy upstream
Error in batch 238 for model llama3-70b-8192: no healthy upstream
Error in batch 239 for model llama3-70b-8192: no healthy upstream
Accuracy for llama3-70b-8192: 0.686 (2836/4133)

=== Final Model Accuracies ===
llama3-70b-8192: 0.686


# Chain of Thought: llama3-70b-8192(Acc=68.7)

In [ ]:
import re
import time
import numpy as np
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
# --- 1. Setup OpenRouter client ---
GROQ_API_KEY = "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"
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=GROQ_API_KEY
)
batch_size = 20
# --- 2. System Prompt ---
SYSTEM_PROMPT_COT = (
    "You are a medical assistant answering MCQs. For each question, think why each option is right or wrong briefly, then pick the most appropriate answer."
    "Return ONLY one answer from A,B,C or D (nothing else should be included in answer), for each {{batch_size}} question STRICTLY IN FORMAT:\n"
    "Q1: A, Q2: C, ..., Q10: B\n"
    "**IMPORTANT**Do not add explanations or extra text."
)

# --- 3. Ask LLM in Batches of 20 ---
def ask_llm_batch_openrouter(questions_batch, model):
    prompt = ""
    for i, row in enumerate(questions_batch, start=1):
        prompt += (

            f"Q{i}: {row['question']}\n"
            f"A. {row['opa']}\n"
            f"B. {row['opb']}\n"
            f"C. {row['opc']}\n"
            f"D. {row['opd']}\n\n"
        )
    prompt += "Write your answers now:\n"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_COT},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=130,
    )

    return response.choices[0].message.content.strip()

# --- 4. Parse answer letters A/B/C/D into 0/1/2/3 ---
def parse_batch_answers(response_text, expected_len):
    matches = re.findall(r"Q\d+:\s*([A-D])", response_text, flags=re.IGNORECASE)
    preds = [ord(m.upper()) - 65 for m in matches[:expected_len]]
    return preds

# --- 5. Load Validation Data ---
validation_df = pd.read_csv('/content/drive/MyDrive/MedDataset/validation.csv')
# validation_df.info()
# validation_df=validation_df.head(200)

# --- 6. Define Models to Evaluate ---
models = [
    "llama3-70b-8192",
]

# --- 7. Batch Evaluation for Each Model ---

results = {}

for model_name in models:
    print(f"\n--- Evaluating {model_name} ---")
    correct, total = 0, 0

    total_batches = (len(validation_df) + batch_size - 1) // batch_size
    for i in tqdm(range(0, len(validation_df), batch_size), desc=f"{model_name} Progress", total=total_batches):

        batch_df = validation_df.iloc[i:i + batch_size]
        golds = batch_df["cop"].astype(int).map({0:'A', 1:'B', 2:'C', 3:'D'}).tolist()

        try:

            raw_response = ask_llm_batch_openrouter(batch_df.to_dict("records"), model=model_name)
            matches = re.findall(r"Q\d+:\s*([A-D])", raw_response, flags=re.IGNORECASE)


            if len(matches) != len(golds):
                print(raw_response)
                print(f"Batch {i//batch_size + 1}: Expected {len(golds)} answers, got {len(matches)}. Skipping.")
                continue

            preds = [m.upper() for m in matches[:len(golds)]]
            correct += sum([int(p == g) for p, g in zip(preds, golds)])
            total += len(golds)

        except Exception as e:
            print(f"Error in batch {i//batch_size + 1} for model {model_name}: {e}")
            continue

    if total > 0:
        acc = correct / total
        results[model_name] = acc
        print(f"Accuracy for {model_name}: {acc:.3f} ({correct}/{total})")
    else:
        print(f"No batches processed for {model_name}")

# --- 8. Summary ---
print("\n=== Final Model Accuracies ===")
for model, acc in results.items():
    print(f"{model}: {acc:.3f}")


--- Evaluating llama3-70b-8192 ---


llama3-70b-8192 Progress:   0%|          | 0/210 [00:00<?, ?it/s]

Q1: A
Q2: A
Q3: B
Q4: D
Q5: B
Q6: A
Q7: D
Q8: B
Q9: D
Q10: A
Q11: A
Q12: None of the above (Note: The options provided are all related to fractures, but the question asks about the most common tumor of the lacrimal gland)
Q13: A
Q14: C
Q15: C
Q16: B
Q17: C
Q18: B
Q19: A
Q20: A
Batch 6: Expected 20 answers, got 19. Skipping.
Error in batch 185 for model llama3-70b-8192: no healthy upstream
Error in batch 186 for model llama3-70b-8192: no healthy upstream
Error in batch 187 for model llama3-70b-8192: no healthy upstream
Error in batch 188 for model llama3-70b-8192: no healthy upstream
Error in batch 189 for model llama3-70b-8192: no healthy upstream
Error in batch 190 for model llama3-70b-8192: no healthy upstream
Error in batch 191 for model llama3-70b-8192: no healthy upstream
Accuracy for llama3-70b-8192: 0.687 (2762/4023)

=== Final Model Accuracies ===
llama3-70b-8192: 0.687


# Tree of Thought: llama3-70b-8192

In [ ]:
import re
import time
import numpy as np
import pandas as pd
from openai import OpenAI

# --- 1. Setup OpenRouter client ---
GROQ_API_KEY = "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"
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=GROQ_API_KEY
)

# --- 2. System Prompt ---
SYSTEM_PROMPT_TOT = (
    "You are a medical assistant evaluating each MCQ with 3 reasoning paths (Path A, B, C). "
    "After evaluating all options under each path, choose the most consistent answer."
    "Return ONLY one the answer from A,B,C or D (nothing else should be included in answer), for each question STRICTLY IN FORMAT:\n"
    "Q1: A, Q2: C, ..., Q10: B\n"
    "**IMPORTANT**Do not add explanations or extra text."
)


# --- 3. Ask LLM in Batches of 20 ---
def ask_llm_batch_openrouter(questions_batch, model):
    prompt = ""
    for i, row in enumerate(questions_batch, start=1):
        prompt += (

            f"Q{i}: {row['question']}\n"
            f"A. {row['opa']}\n"
            f"B. {row['opb']}\n"
            f"C. {row['opc']}\n"
            f"D. {row['opd']}\n\n"
        )
    prompt += "Write your answers now:\n"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_TOT},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
    )

    return response.choices[0].message.content.strip()

# --- 4. Parse answer letters A/B/C/D into 0/1/2/3 ---
def parse_batch_answers(response_text, expected_len):
    matches = re.findall(r"Q\d+:\s*([A-D])", response_text, flags=re.IGNORECASE)
    preds = [ord(m.upper()) - 65 for m in matches[:expected_len]]
    return preds

# --- 5. Load Validation Data ---
validation_df = pd.read_csv('/content/drive/MyDrive/MedDataset/validation.csv')
# validation_df.info()
# validation_df=validation_df.head(200)

# --- 6. Define Models to Evaluate ---
models = [
    "llama3-70b-8192",
]

# --- 7. Batch Evaluation for Each Model ---
batch_size = 10
results = {}

for model_name in models:
    print(f"\n--- Evaluating {model_name} ---")
    correct, total = 0, 0

    total_batches = (len(validation_df) + batch_size - 1) // batch_size
    for i in tqdm(range(0, len(validation_df), batch_size), desc=f"{model_name} Progress", total=total_batches):

        batch_df = validation_df.iloc[i:i + batch_size]
        golds = batch_df["cop"].astype(int).map({0:'A', 1:'B', 2:'C', 3:'D'}).tolist()

        try:

            raw_response = ask_llm_batch_openrouter(batch_df.to_dict("records"), model=model_name)
            matches = re.findall(r"Q\d+:\s*([A-D])", raw_response, flags=re.IGNORECASE)

            if len(matches) != len(golds):
                print(raw_response)
                print(f"Batch {i//batch_size + 1}: Expected {len(golds)} answers, got {len(matches)}. Skipping.")
                continue

            preds = [m.upper() for m in matches[:len(golds)]]
            correct += sum([int(p == g) for p, g in zip(preds, golds)])
            total += len(golds)

        except Exception as e:
            print(f"Error in batch {i//batch_size + 1} for model {model_name}: {e}")
            continue

    if total > 0:
        acc = correct / total
        results[model_name] = acc
        print(f"Accuracy for {model_name}: {acc:.3f} ({correct}/{total})")
    else:
        print(f"No batches processed for {model_name}")

# --- 8. Summary ---
print("\n=== Final Model Accuracies ===")
for model, acc in results.items():
    print(f"{model}: {acc:.3f}")


--- Evaluating llama3-70b-8192 ---


llama3-70b-8192 Progress:   0%|          | 0/419 [00:00<?, ?it/s]

Error in batch 1 for model llama3-70b-8192: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jvp6btraetj9pg8agvw334je` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500043, Requested 736. Please try again in 2m14.7308s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error in batch 2 for model llama3-70b-8192: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jvp6btraetj9pg8agvw334je` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500043, Requested 652. Please try again in 2m0.1376s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error in batch 3 for model llama3-70b-8192: Error code: 429 - {'error': {'message': 'Rate limit reached for m

# **OpenRouter**

# Zero Shot: microsoft/mai-ds-r1:free(Acc=76.3)

In [ ]:
import re
import time
import numpy as np
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm

# --- 1. Setup OpenRouter client ---
OPEN_ROUTER_KEY = "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"
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPEN_ROUTER_KEY,
)

# --- 2. System Prompt ---

SYSTEM_PROMPT = (
    "You are a medical examination assistant. "
    "For every batch you will receive exactly <=10 multiple-choice questions (MCQs) in the format:\n\n"
    "Q1: <question>\nA. <option>\nB. <option>\nC. <option>\nD. <option>\n\n"
    "Return ONLY one the answer from A,B,C or D (nothing else should be included in answer), for each question STRICTLY IN FORMAT:\n"
    "Q1: A, Q2: C, ..., Q20: B\n"
    "**IMPORTANT**Do not add explanations or extra text."
)



# --- 3. Ask LLM in Batches of 20 ---
def ask_llm_batch_openrouter(questions_batch, model):
    prompt = ""
    for i, row in enumerate(questions_batch, start=1):
        prompt += (

            f"Q{i}: {row['question']}\n"
            f"A. {row['opa']}\n"
            f"B. {row['opb']}\n"
            f"C. {row['opc']}\n"
            f"D. {row['opd']}\n\n"
        )
    prompt += "Write your answers now:\n"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        top_p=0.95,
    )

    return response.choices[0].message.content.strip()

# --- 4. Parse answer letters A/B/C/D into 0/1/2/3 ---
def parse_batch_answers(response_text, expected_len):
    matches = re.findall(r"Q\d+:\s*([A-D])", response_text, flags=re.IGNORECASE)
    preds = [ord(m.upper()) - 65 for m in matches[:expected_len]]
    return preds

# --- 5. Load Validation Data ---
validation_df = pd.read_csv('/content/drive/MyDrive/MedDataset/validation.csv')
# validation_df.info()
# validation_df=validation_df.head(100)

# --- 6. Define Models to Evaluate ---
models = [
    "microsoft/mai-ds-r1:free",
]

# --- 7. Batch Evaluation for Each Model ---
batch_size = 40
results = {}

for model_name in models:
    print(f"\n--- Evaluating {model_name} ---")
    correct, total = 0, 0

    total_batches = (len(validation_df) + batch_size - 1) // batch_size
    for i in tqdm(range(0, len(validation_df), batch_size), desc=f"{model_name} Progress", total=total_batches):

        batch_df = validation_df.iloc[i:i + batch_size]
        golds = batch_df["cop"].astype(int).map({0:'A', 1:'B', 2:'C', 3:'D'}).tolist()

        try:

            raw_response = ask_llm_batch_openrouter(batch_df.to_dict("records"), model=model_name)
            matches = re.findall(r"Q\d+:\s*([A-D])", raw_response, flags=re.IGNORECASE)

            if len(matches) != len(golds):
                print(raw_response)
                print(f"Batch {i//batch_size + 1}: Expected {len(golds)} answers, got {len(matches)}. Skipping.")
                continue

            preds = [m.upper() for m in matches[:len(golds)]]
            correct += sum([int(p == g) for p, g in zip(preds, golds)])
            total += len(golds)

        except Exception as e:
            print(f"Error in batch {i//batch_size + 1} for model {model_name}: {e}")
            continue

    if total > 0:
        acc = correct / total
        results[model_name] = acc
        print(f"Accuracy for {model_name}: {acc:.3f} ({correct}/{total})")
    else:
        print(f"No batches processed for {model_name}")

# --- 8. Summary ---
print("\n=== Final Model Accuracies ===")
for model, acc in results.items():
    print(f"{model}: {acc:.3f}")


--- Evaluating microsoft/mai-ds-r1:free ---


microsoft/mai-ds-r1:free Progress:   0%|          | 0/105 [00:00<?, ?it/s]

Accuracy for microsoft/mai-ds-r1:free: 0.763 (3193/4183)

=== Final Model Accuracies ===
microsoft/mai-ds-r1:free: 0.763


# Zero Shot: Gemini 1.5 flash(Acc=54.9), openai/o4-mini(Acc=80.1)


In [ ]:
import re
import time
import numpy as np
import pandas as pd
from openai import OpenAI

# --- 1. Setup OpenRouter client ---
OPEN_ROUTER_KEY = "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"
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPEN_ROUTER_KEY,
)

# --- 2. System Prompt ---
SYSTEM_PROMPT = (
    "You are a medical examination assistant. "
    "For every batch you will receive exactly 20 multiple-choice questions (MCQs) in the format:\n\n"
    "Q1: <question>\nA. <option>\nB. <option>\nC. <option>\nD. <option>\n\n"
    "Return ONLY one the answer from A,B,C or D (nothing else should be included in answer), for each question STRICTLY IN FORMAT:\n"
    "Q1: A, Q2: C, ..., Q10: B\n"
    "**IMPORTANT**Do not add explanations or extra text."
)

# --- 3. Ask LLM in Batches of 20 ---
def ask_llm_batch_openrouter(questions_batch, model):
    prompt = ""
    for i, row in enumerate(questions_batch, start=1):
        prompt += (

            f"Q{i}: {row['question']}\n"
            f"A. {row['opa']}\n"
            f"B. {row['opb']}\n"
            f"C. {row['opc']}\n"
            f"D. {row['opd']}\n\n"
        )
    prompt += "Write your answers now:\n"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        top_p=0.95,
    )

    return response.choices[0].message.content.strip()

# --- 4. Parse answer letters A/B/C/D into 0/1/2/3 ---
def parse_batch_answers(response_text, expected_len):
    matches = re.findall(r"Q\d+:\s*([A-D])", response_text, flags=re.IGNORECASE)
    preds = [ord(m.upper()) - 65 for m in matches[:expected_len]]
    return preds

# --- 5. Load Validation Data ---
validation_df = pd.read_csv('/content/drive/MyDrive/MedDataset/validation.csv')
# validation_df.info()
# validation_df=validation_df.head(200)

# --- 6. Define Models to Evaluate ---
models = [
    "google/gemini-flash-1.5-8b",
    "openai/o4-mini",

]

# --- 7. Batch Evaluation for Each Model ---
batch_size = 10
results = {}

for model_name in models:
    print(f"\n--- Evaluating {model_name} ---")
    correct, total = 0, 0

    total_batches = (len(validation_df) + batch_size - 1) // batch_size
    for i in tqdm(range(0, len(validation_df), batch_size), desc=f"{model_name} Progress", total=total_batches):

        batch_df = validation_df.iloc[i:i + batch_size]
        golds = batch_df["cop"].astype(int).map({0:'A', 1:'B', 2:'C', 3:'D'}).tolist()

        try:

            raw_response = ask_llm_batch_openrouter(batch_df.to_dict("records"), model=model_name)
            matches = re.findall(r"Q\d+:\s*([A-D])", raw_response, flags=re.IGNORECASE)

            if len(matches) != len(golds):
                print(raw_response)
                print(f"Batch {i//batch_size + 1}: Expected {len(golds)} answers, got {len(matches)}. Skipping.")
                continue

            preds = [m.upper() for m in matches[:len(golds)]]
            correct += sum([int(p == g) for p, g in zip(preds, golds)])
            total += len(golds)

        except Exception as e:
            print(f"Error in batch {i//batch_size + 1} for model {model_name}: {e}")
            continue

    if total > 0:
        acc = correct / total
        results[model_name] = acc
        print(f"Accuracy for {model_name}: {acc:.3f} ({correct}/{total})")
    else:
        print(f"No batches processed for {model_name}")

# --- 8. Summary ---
print("\n=== Final Model Accuracies ===")
for model, acc in results.items():
    print(f"{model}: {acc:.3f}")


--- Evaluating google/gemini-flash-1.5-8b ---


google/gemini-flash-1.5-8b Progress:   0%|          | 0/419 [00:00<?, ?it/s]

Accuracy for google/gemini-flash-1.5-8b: 0.549 (2295/4183)

--- Evaluating openai/o4-mini ---


openai/o4-mini Progress:   0%|          | 0/419 [00:00<?, ?it/s]


Batch 183: Expected 10 answers, got 0. Skipping.

Batch 392: Expected 10 answers, got 0. Skipping.
Accuracy for openai/o4-mini: 0.801 (3336/4163)

--- Evaluating deepseek/deepseek-r1-0528:free ---


deepseek/deepseek-r1-0528:free Progress:   0%|          | 0/419 [00:00<?, ?it/s]

# Chain of Thought: Gemini 1.5 flash(Acc=55.6)

In [ ]:
import re
import time
import numpy as np
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm

# --- 1. Setup OpenRouter client ---
OPEN_ROUTER_KEY = "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"
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPEN_ROUTER_KEY,
)

# --- 2. System Prompt ---

SYSTEM_PROMPT_COT = (
    "You are a medical assistant answering MCQs. For each question, explain why each option is right or wrong briefly, then pick the most appropriate answer."
    "Return ONLY one the answer from A,B,C or D (nothing else should be included in answer), for each question STRICTLY IN FORMAT:\n"
    "Q1: A, Q2: C, ..., Q10: B\n"
    "**IMPORTANT**Do not add explanations or extra text."
)



# --- 3. Ask LLM in Batches of 20 ---
def ask_llm_batch_openrouter(questions_batch, model):
    prompt = ""
    for i, row in enumerate(questions_batch, start=1):
        prompt += (

            f"Q{i}: {row['question']}\n"
            f"A. {row['opa']}\n"
            f"B. {row['opb']}\n"
            f"C. {row['opc']}\n"
            f"D. {row['opd']}\n\n"
        )
    prompt += "Write your answers now:\n"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_COT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        top_p=0.95,
    )

    return response.choices[0].message.content.strip()

# --- 4. Parse answer letters A/B/C/D into 0/1/2/3 ---
def parse_batch_answers(response_text, expected_len):
    matches = re.findall(r"Q\d+:\s*([A-D])", response_text, flags=re.IGNORECASE)
    preds = [ord(m.upper()) - 65 for m in matches[:expected_len]]
    return preds

# --- 5. Load Validation Data ---
validation_df = pd.read_csv('/content/drive/MyDrive/MedDataset/validation.csv')
# validation_df.info()
# validation_df=validation_df.head(100)

# --- 6. Define Models to Evaluate ---
models = [
    "google/gemini-flash-1.5-8b",
]

# --- 7. Batch Evaluation for Each Model ---
batch_size = 10
results = {}

for model_name in models:
    print(f"\n--- Evaluating {model_name} ---")
    correct, total = 0, 0

    total_batches = (len(validation_df) + batch_size - 1) // batch_size
    for i in tqdm(range(0, len(validation_df), batch_size), desc=f"{model_name} Progress", total=total_batches):

        batch_df = validation_df.iloc[i:i + batch_size]
        golds = batch_df["cop"].astype(int).map({0:'A', 1:'B', 2:'C', 3:'D'}).tolist()

        try:

            raw_response = ask_llm_batch_openrouter(batch_df.to_dict("records"), model=model_name)
            matches = re.findall(r"Q\d+:\s*([A-D])", raw_response, flags=re.IGNORECASE)

            if len(matches) != len(golds):
                print(raw_response)
                print(f"Batch {i//batch_size + 1}: Expected {len(golds)} answers, got {len(matches)}. Skipping.")
                continue

            preds = [m.upper() for m in matches[:len(golds)]]
            correct += sum([int(p == g) for p, g in zip(preds, golds)])
            total += len(golds)

        except Exception as e:
            print(f"Error in batch {i//batch_size + 1} for model {model_name}: {e}")
            continue

    if total > 0:
        acc = correct / total
        results[model_name] = acc
        print(f"Accuracy for {model_name}: {acc:.3f} ({correct}/{total})")
    else:
        print(f"No batches processed for {model_name}")

# --- 8. Summary ---
print("\n=== Final Model Accuracies ===")
for model, acc in results.items():
    print(f"{model}: {acc:.3f}")


--- Evaluating google/gemini-flash-1.5-8b ---


google/gemini-flash-1.5-8b Progress:   0%|          | 0/419 [00:00<?, ?it/s]

Accuracy for google/gemini-flash-1.5-8b: 0.556 (2325/4183)

--- Evaluating deepseek/deepseek-r1-0528:free ---


deepseek/deepseek-r1-0528:free Progress:   0%|          | 0/419 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Tree of Thought: Gemini 1.5 flash(Acc=55.0)

In [ ]:
import re
import time
import numpy as np
import pandas as pd
from openai import OpenAI

# --- 1. Setup OpenRouter client ---
OPEN_ROUTER_KEY = "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"
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPEN_ROUTER_KEY,
)

# --- 2. System Prompt ---
SYSTEM_PROMPT_TOT = (
    "You are a medical assistant evaluating each MCQ with 3 reasoning paths (Path A, B, C). "
    "After evaluating all options under each path, choose the most consistent answer."
    "Return ONLY one the answer from A,B,C or D (nothing else should be included in answer), for each question STRICTLY IN FORMAT:\n"
    "Q1: A, Q2: C, ..., Q10: B\n"
    "**IMPORTANT**Do not add explanations or extra text."
)


# --- 3. Ask LLM in Batches of 20 ---
def ask_llm_batch_openrouter(questions_batch, model):
    prompt = ""
    for i, row in enumerate(questions_batch, start=1):
        prompt += (

            f"Q{i}: {row['question']}\n"
            f"A. {row['opa']}\n"
            f"B. {row['opb']}\n"
            f"C. {row['opc']}\n"
            f"D. {row['opd']}\n\n"
        )
    prompt += "Write your answers now:\n"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_TOT},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        top_p=0.95,
    )

    return response.choices[0].message.content.strip()

# --- 4. Parse answer letters A/B/C/D into 0/1/2/3 ---
def parse_batch_answers(response_text, expected_len):
    matches = re.findall(r"Q\d+:\s*([A-D])", response_text, flags=re.IGNORECASE)
    preds = [ord(m.upper()) - 65 for m in matches[:expected_len]]
    return preds

# --- 5. Load Validation Data ---
validation_df = pd.read_csv('/content/drive/MyDrive/MedDataset/validation.csv')
# validation_df.info()
# validation_df=validation_df.head(100)

# --- 6. Define Models to Evaluate ---
models = [
    "google/gemini-flash-1.5-8b",

]

# --- 7. Batch Evaluation for Each Model ---
batch_size = 10
results = {}

for model_name in models:
    print(f"\n--- Evaluating {model_name} ---")
    correct, total = 0, 0

    total_batches = (len(validation_df) + batch_size - 1) // batch_size
    for i in tqdm(range(0, len(validation_df), batch_size), desc=f"{model_name} Progress", total=total_batches):

        batch_df = validation_df.iloc[i:i + batch_size]
        golds = batch_df["cop"].astype(int).map({0:'A', 1:'B', 2:'C', 3:'D'}).tolist()

        try:

            raw_response = ask_llm_batch_openrouter(batch_df.to_dict("records"), model=model_name)
            matches = re.findall(r"Q\d+:\s*([A-D])", raw_response, flags=re.IGNORECASE)

            if len(matches) != len(golds):
                print(raw_response)
                print(f"Batch {i//batch_size + 1}: Expected {len(golds)} answers, got {len(matches)}. Skipping.")
                continue

            preds = [m.upper() for m in matches[:len(golds)]]
            correct += sum([int(p == g) for p, g in zip(preds, golds)])
            total += len(golds)

        except Exception as e:
            print(f"Error in batch {i//batch_size + 1} for model {model_name}: {e}")
            continue

    if total > 0:
        acc = correct / total
        results[model_name] = acc
        print(f"Accuracy for {model_name}: {acc:.3f} ({correct}/{total})")
    else:
        print(f"No batches processed for {model_name}")

# --- 8. Summary ---
print("\n=== Final Model Accuracies ===")
for model, acc in results.items():
    print(f"{model}: {acc:.3f}")


--- Evaluating google/gemini-flash-1.5-8b ---


google/gemini-flash-1.5-8b Progress:   0%|          | 0/419 [00:00<?, ?it/s]

Accuracy for google/gemini-flash-1.5-8b: 0.550 (2301/4183)

--- Evaluating openai/o4-mini ---


openai/o4-mini Progress:   0%|          | 0/419 [00:00<?, ?it/s]

KeyboardInterrupt: 